# Data Cleaning 

In [3]:
import pandas as pd 
import re 

In [5]:
df_historical_data = pd.read_csv("Desktop/euro_historical_data.csv")   # Historical Data from 1960-2020
df_euro = pd.read_csv("Desktop/euro_2024_fixtures.csv")   # fixtures 2024

In [7]:
df_historical_data 

,home,score,away,Year
0,France,4–5,Yugoslavia,1960
1,Czechoslovakia,0–3,Soviet Union,1960
2,Czechoslovakia,2–0,France,1960
3,Soviet Union,2–1 (a.e.t.),Yugoslavia,1960
4,Spain,2–1 (a.e.t.),Hungary,1964
...,...,...,...,...
332,Czech Republic,1–2,Denmark,2020
333,Ukraine,0–4,England,2020
334,Italy,1–1 (a.e.t.),Spain,2020
335,England,2–1 (a.e.t.),Denmark,2020


In [11]:
df_euro.head(10)

,home,away,Year,Match
0,Germany,Scotland,2024,1
1,Hungary,Switzerland,2024,2
2,Germany,Hungary,2024,3
3,Scotland,Switzerland,2024,4
4,Switzerland,Germany,2024,5
5,Scotland,Hungary,2024,6
6,Spain,Croatia,2024,7
7,Italy,Albania,2024,8
8,Croatia,Albania,2024,9
9,Spain,Italy,2024,10


## Cleaning df_historical_data 

In [17]:
# check if there is any null value use isnull(), stack() and index(). Stack() converts dataFrame to series and keep only the True Values
missing_indices = df_historical_data.isnull().stack().index[df_historical_data.isnull().stack()]
print(list(missing_indices))

[]


In [27]:
# Cleaning Home Away and score coloumn

# Remove whitespaces
df_historical_data['home'] = df_historical_data['home'].str.strip()
df_historical_data['away'] = df_historical_data['away'].str.strip()

# Define a function to extract scores
def extract_score(score):
    # Extract the score part using a regular expression
    match = re.match(r'(\d+)[^\d]*(\d+)', str(score))
    if match:
        return f"{match.group(1)}–{match.group(2)}"
    else:
        return '0–0'  # Default score if no match is found

# Apply the function to the 'score' column
df_historical_data['score'] = df_historical_data['score'].apply(extract_score)

# Splitting 'score' into 'HomeGoals' and 'AwayGoals' columns
df_historical_data[['HomeGoals', 'AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)

# Convert the goals columns to integers
df_historical_data['HomeGoals'] = df_historical_data['HomeGoals'].astype(int)
df_historical_data['AwayGoals'] = df_historical_data['AwayGoals'].astype(int)
df_historical_data['Year'] = df_historical_data['Year'].astype(int)
df_historical_data


,home,score,away,Year,HomeGoals,AwayGoals
0,France,4–5,Yugoslavia,1960,4,5
1,Czechoslovakia,0–3,Soviet Union,1960,0,3
2,Czechoslovakia,2–0,France,1960,2,0
3,Soviet Union,2–1,Yugoslavia,1960,2,1
4,Spain,2–1,Hungary,1964,2,1
...,...,...,...,...,...,...
332,Czech Republic,1–2,Denmark,2020,1,2
333,Ukraine,0–4,England,2020,0,4
334,Italy,1–1,Spain,2020,1,1
335,England,2–1,Denmark,2020,2,1


In [29]:
# Rename Columns 
df_historical_data.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam'}, inplace=True)

# Making Total Goals Column
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']
df_historical_data

,HomeTeam,score,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,4–5,Yugoslavia,1960,4,5,9
1,Czechoslovakia,0–3,Soviet Union,1960,0,3,3
2,Czechoslovakia,2–0,France,1960,2,0,2
3,Soviet Union,2–1,Yugoslavia,1960,2,1,3
4,Spain,2–1,Hungary,1964,2,1,3
...,...,...,...,...,...,...,...
332,Czech Republic,1–2,Denmark,2020,1,2,3
333,Ukraine,0–4,England,2020,0,4,4
334,Italy,1–1,Spain,2020,1,1,2
335,England,2–1,Denmark,2020,2,1,3


In [31]:
# Dropping Score Column 
df_historical_data.drop('score', axis=1, inplace=True)
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Yugoslavia,1960,4,5,9
1,Czechoslovakia,Soviet Union,1960,0,3,3
2,Czechoslovakia,France,1960,2,0,2
3,Soviet Union,Yugoslavia,1960,2,1,3
4,Spain,Hungary,1964,2,1,3
...,...,...,...,...,...,...
332,Czech Republic,Denmark,2020,1,2,3
333,Ukraine,England,2020,0,4,4
334,Italy,Spain,2020,1,1,2
335,England,Denmark,2020,2,1,3


## Cleaning df_Euro

In [33]:
df_euro['home'] = df_euro['home'].str.strip()
df_euro['away'] = df_euro['away'].str.strip()              

## Exporting Clean DataFrames 

In [35]:
df_historical_data.to_csv('clean_euro_cup_matches.csv', index=False)
df_euro.to_csv('clean_euro_cup_fixtures.csv', index=False)

## Extra Verification 

In [39]:
years = [1960,1964,1968,1972,1976,1980,1984,1988,1992,1996,2000,2004,2008,2012,2016,2020]
for year in years:
    print(year, len(df_historical_data[df_historical_data['Year']==year])) # to check number of matches every year

1960 4
1964 4
1968 5
1972 4
1976 4
1980 14
1984 15
1988 15
1992 15
1996 31
2000 31
2004 31
2008 31
2012 31
2016 51
2020 51


In [45]:
# verify data collected for a team
print(df_historical_data[df_historical_data['HomeTeam'].str.contains('Ukraine')])
print(df_historical_data[df_historical_data['AwayTeam'].str.contains('Ukraine')])

    HomeTeam          AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
223  Ukraine            Sweden  2012          2          1           3
224  Ukraine            France  2012          0          2           2
249  Ukraine  Northern Ireland  2016          0          2           2
251  Ukraine            Poland  2016          0          1           1
300  Ukraine   North Macedonia  2020          2          1           3
303  Ukraine           Austria  2020          0          1           1
333  Ukraine           England  2020          0          4           4
        HomeTeam AwayTeam  Year  HomeGoals  AwayGoals  TotalGoals
226      England  Ukraine  2012          1          0           1
248      Germany  Ukraine  2016          2          0           2
299  Netherlands  Ukraine  2020          3          2           5
329       Sweden  Ukraine  2020          1          2           3
